# EMMO Browser example
This notebook is an example on how to utilize triples to generate an ontology documentation browser

## Installation
    $ pip install ipytree
    
    If you use JupyterLab<=2:
    $ jupyter labextension install @jupyter-widgets/jupyterlab-manager ipytree

    If you have notebook 5.2 or below, you also need to execute:
    $ jupyter nbextension enable --py --sys-prefix ipytree

In [1]:
from typing import Tuple, List, Dict
from dataclasses import dataclass

# Our humble triplestore
triples: List[Tuple[str, str, str]] = []

# Helper functions for perfoming triple matching
create_match = lambda tr: lambda s, p, o: [t for t in tr if (not s or t[0] == s) and (not p or t[1] == p) and (not o or t[2] == o)]
match = lambda s = None, p = None, o = None: create_match(triples)(s, p, o)
match_first = lambda s = None, p = None, o = None : next(iter(match(s, p, o) or []), (None, None, None))


In [2]:
# Import triples onto the triplestore
import shlex 
    
filenames = ['emmo.nt']
for filename in filenames:
    with open(filename, 'r') as file:            
        lines = shlex.split(file.read());        
        for i in range(0, len(lines), 4):
            triples.append((lines[i], lines[i+1], lines[i+2])) 
          

In [3]:
from ipytree import Tree, Node
from ipywidgets import HBox, HTML, Output, Layout
from jinja2 import Template
import html
from namespaces import RDFS, SKOS, EMMO

# Read the jinja2-template from file
with open("card.html.j2", "r") as f:
    template = f.read()
    jtemplate = Template(template)

# Declare the output window for the concept documentation
documentation = Output(layout=Layout(min_width='50%'))

# Helper-map that maps ipytree node-uuids to concept-URI
nodemap = {}

def match_to_string(concept, predicate):
    """ Concatenate and wash matching object strings """
    return "".join([(html.unescape(text))
                    .replace('@en', '\n')
                    .replace('\\n', '<br/>\n') 
                    for (_,_,text) in match(concept, predicate)])

def handle_click(event):
    """ Event to be called when a node is clicked """
    concept = nodemap[event['owner']._id]
    documentation.clear_output()
    with documentation:        
        display(HTML(value=jtemplate.render(
            label=event['owner'].name, 
            concept=html.escape(concept), 
            elucidation=match_to_string(concept, EMMO.elucidation),
            comments=match_to_string(concept, RDFS.comment), 
            etymology=match_to_string(concept, EMMO.etymology))))
        
        
def generate_node_tree(root):
    """ Recursively generate the ipytree nodes """
    sub_nodes = []
    
    # Recursively generate and append sub-nodes
    for (sub_node,_,_) in match(None, RDFS.subClassOf, root):        
        sub_nodes.append(generate_node_tree(sub_node))
        
    # Find the prefLabel (node name)
    (_,_,lbl) = match_first(root, SKOS.prefLabel, None)
    lbl = lbl.replace('@en','')
    if sub_nodes:
        node = Node(lbl, sub_nodes, opened=False, icon_style="success")
    else:
        node = Node(lbl, sub_nodes, opened=False, icon="leaf", icon_style="success")
        
    # Add observer to handle interactions
    node.observe(handle_click, 'selected')
    nodemap[node._id] = root
    return node

# Generate the node tree
emmo_node = generate_node_tree(EMMO.Emmo)

In [4]:
tree=Tree(layout=Layout(min_width='50%'))
tree.add_node(emmo_node)
box = HBox(children=[tree, documentation], layout=Layout(align_items='stretch', width='100%'))
display (box)
